### OpenAI API

In [ ]:
# SetUp library
%pip install -U openai pydantic python-dotenv

import os, sys, openai

print("Python:", sys.version)
print("openai SDK:", openai.__version__)
print("Kernel:", sys.executable)
print("Has OPENAI_API_KEY?:", "OPENAI_API_KEY" in os.environ)

In [ ]:
# Check API_KEY
import os

print("OPENAI_API_KEY:", os.environ.get("OPENAI_API_KEY", "Not set"))

### (1) responses.create：一般的な応答生成

In [ ]:
prompt = """以下の要件で簡潔に回答してください：
- 対象: Python学習者
- トピック: OpenAI APIのはじめ方
- 箇条書き3つ
"""
from openai import OpenAI

# client = OpenAI()
client = OpenAI(api_key=os.environ["OPENAI_API_KEY"])

MODEL = "gpt-5-mini"

resp = client.responses.create(
    model=MODEL,
    input=prompt,
)

print(resp.output_text)  # テキスト抽出の便利プロパティ

#### (2) responses.parse：構造化出力（Pydantic）

In [ ]:
# responses.parse：構造化出力（Pydantic）
from dotenv import load_dotenv
load_dotenv()

from openai import OpenAI
from pydantic import BaseModel, Field
from typing import List
import json, inspect, os

client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))
MODEL = os.getenv("OPENAI_MODEL", "gpt-5-mini")  # 環境変数で上書き可

class TodoItem(BaseModel):
    title: str = Field(..., description="やることのタイトル")
    priority: int = Field(..., ge=1, le=5, description="1(低)〜5(高)")
    tags: List[str] = Field(default_factory=list)

class TodoPlan(BaseModel):
    owner: str
    items: List[TodoItem]

instruction = "あなたは秘書です。日本語で、学習計画のToDoを構造化して返してください。"
user_text  = "明日はOpenAIのResponses APIを勉強、週末はQdrantを触る。優先度高いのは明日の学習。タグは['api','vector']"

def to_json_schema(model_cls):
    # pydantic v2 / v1 どちらでもOKにする
    if hasattr(model_cls, "model_json_schema"):
        return model_cls.model_json_schema()
    return model_cls.schema()

def parse_todo_plan():
    # 1) 新SDK: responses.parse(text_format=MyModel)
    try:
        sig = inspect.signature(client.responses.parse)  # 存在確認
        if "text_format" in sig.parameters:
            resp = client.responses.parse(
                model=MODEL,
                instructions=instruction,
                input=user_text,
                text_format=TodoPlan,    # ← ココがポイント（schema ではない）
            )
            return getattr(resp, "output_parsed", getattr(resp, "parsed", resp))
    except Exception as _:
        pass

    # 2) 構造化出力: responses.create(response_format=json_schema)
    try:
        schema = to_json_schema(TodoPlan)
        resp = client.responses.create(
            model=MODEL,
            instructions=instruction,
            input=user_text,
            response_format={
                "type": "json_schema",
                "json_schema": {"name": "TodoPlan", "schema": schema, "strict": True},
            },
        )
        data = json.loads(resp.output_text)
        return TodoPlan.model_validate(data)
    except TypeError:
        # 古いSDKだと response_format 未対応 → 3) chat.completions(JSONモード)へ
        pass

    # 3) 互換: Chat Completions + JSONモード → Pydanticで検証
    chat = client.chat.completions.create(
        model=MODEL,  # 4o/5-mini等のJSONモード対応モデルを推奨
        messages=[
            {"role": "system", "content": "必ず有効なJSONのみを出力してください。"},
            {"role": "user", "content": f"{instruction}\nユーザー入力: {user_text}\n出力は次のスキーマに厳密に従ってください: {to_json_schema(TodoPlan)}"},
        ],
        response_format={"type": "json_object"},
        temperature=0,
    )
    data = json.loads(chat.choices[0].message.content)
    return TodoPlan.model_validate(data)

parsed_plan = parse_todo_plan()
print(parsed_plan)

### (3) audio.speech.create：テキスト → 音声（TTS）

In [ ]:
# audio.speech.create
from pathlib import Path
from IPython.display import Audio, display

speech_path = Path("sample_ja.mp3")

# 推奨の軽量TTSモデル例（ドキュメントに基づく）
with client.audio.speech.with_streaming_response.create(
    model="gpt-4o-mini-tts",
    voice="alloy",  # 他にverse, aria 等のバリエーションがある場合があります
    input="こんにちは。今日はOpenAI APIの学習、何から始めますか？",
) as resp:
    resp.stream_to_file(speech_path)

display(Audio(filename=str(speech_path)))

### (4) audio.transcriptions.create：音声 → テキスト（日本語の文字起こし）

In [ ]:
audio_file_path = "sample_ja.mp3"  # 例: 日本語音声ファイル

with open(audio_file_path, "rb") as f:
    transcript = client.audio.transcriptions.create(
        model="gpt-4o-mini-transcribe",  # 速く安価なSTTモデル
        file=f,
        response_format="text",          # "verbose_json" なども可
    )

print(transcript.text if hasattr(transcript, "text") else transcript)

### (5) audio.translations.create：音声 → 英語テキスト（翻訳）

In [ ]:
# translation to English
from openai import OpenAI
client = OpenAI()

audio_file_path = "sample_ja.mp3"

with open(audio_file_path, "rb") as f:
    # 音声を英語に翻訳するには translations.create() を使用
    translation = client.audio.translations.create(
        model="whisper-1",  # translations では whisper-1 を使用
        file=f,
        response_format="text",  # 文字列を直接返す
    )

print(translation)  # response_format="text" の場合は str

### (6) images.generate：画像生成（DALL·E系 / gpt-image-1）

In [ ]:
import base64
from IPython.display import Image, display

prompt = "桜と富士山、夕焼け、浮世絵風のイラスト。高解像度で。"

img = client.images.generate(
    model="dall-e-3",  # gpt-image-1 の代わりに dall-e-3 を使用
    prompt=prompt,
    size="1024x1024",
    n=1,
    response_format="b64_json",  # base64形式で取得
)

b64 = img.data[0].b64_json
binary = base64.b64decode(b64)

out_path = "fuji_sakura.png"
with open(out_path, "wb") as f:
    f.write(binary)

display(Image(filename=out_path))

### (7) chat.completions.create：チャット完了API

In [ ]:
chat = client.chat.completions.create(
    model=MODEL,
    messages=[
        {"role": "system", "content": "あなたは有能なPythonメンターです。"},
        {"role": "user", "content": "OpenAI APIでJSONの構造化出力を行うには？"},
    ],
    # temperature=0.2,  # gpt-5-miniはデフォルト値(1)のみサポート
)

print(chat.choices[0].message.content)